In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid

In [2]:
train_url  = "https://raw.githubusercontent.com/Rahulsharma869/Housing-Price-Prediction/main/Datasets/train.csv"
train_set = pd.read_csv(train_url)
train_set

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [3]:
xtrain_col = ["MSSubClass",
             "LotArea",
             "OverallQual",
             "OverallCond",
             "YearBuilt",
             "YearRemodAdd",
             "TotalBsmtSF",
             "1stFlrSF",
             "2ndFlrSF",
             "GrLivArea",
             "BedroomAbvGr",
             "KitchenAbvGr",
             "GarageCars",
             "GarageArea",
             "PoolArea",
             "MoSold",
             "PoolArea",
             "YrSold"]
xtrain = train_set.loc[:,xtrain_col]
ytrain = train_set.loc[:,"SalePrice"]
small_nonzero = 0.001
#xtrain = np.where(xtrain == 0, small_nonzero, xtrain)
print(f'xtrain : {xtrain.shape}')
print(f'ytrain : {ytrain.shape}')
#print(xtrain)
#print(ytrain)
split = int(1468*0.8)
x_train = np.array(xtrain[:split])
y_train = np.array(ytrain[:split])
x_test = np.array(xtrain[split:])
y_test = np.array(ytrain[split:])
xtrain

xtrain : (1460, 18)
ytrain : (1460,)


,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,BedroomAbvGr,KitchenAbvGr,GarageCars,GarageArea,PoolArea,MoSold,PoolArea,YrSold
0,60,8450,7,5,2003,2003,856,856,854,1710,3,1,2,548,0,2,0,2008
1,20,9600,6,8,1976,1976,1262,1262,0,1262,3,1,2,460,0,5,0,2007
2,60,11250,7,5,2001,2002,920,920,866,1786,3,1,2,608,0,9,0,2008
3,70,9550,7,5,1915,1970,756,961,756,1717,3,1,3,642,0,2,0,2006
4,60,14260,8,5,2000,2000,1145,1145,1053,2198,4,1,3,836,0,12,0,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,7917,6,5,1999,2000,953,953,694,1647,3,1,2,460,0,8,0,2007
1456,20,13175,6,6,1978,1988,1542,2073,0,2073,3,1,2,500,0,2,0,2010
1457,70,9042,7,9,1941,2006,1152,1188,1152,2340,4,1,1,252,0,5,0,2010
1458,20,9717,5,6,1950,1996,1078,1078,0,1078,2,1,1,240,0,4,0,2010


In [4]:
from tensorflow.keras import regularizers
model = Sequential(
    [               
        Dense(20, activation='relu', name = 'layer1',input_shape=(18,),kernel_regularizer=regularizers.l2(0.01)),
        Dense(10, activation='relu', name = 'layer2',kernel_regularizer=regularizers.l2(0.01)),
        Dense(5, activation='relu', name = 'layer3',kernel_regularizer=regularizers.l2(0.01)),
        Dense(1, activation='relu', name = 'layer4')
    ], name = "my_model" 
)
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 20)                380       
                                                                 
 layer2 (Dense)              (None, 10)                210       
                                                                 
 layer3 (Dense)              (None, 5)                 55        
                                                                 
 layer4 (Dense)              (None, 1)                 6         
                                                                 
Total params: 651
Trainable params: 651
Non-trainable params: 0
_________________________________________________________________


In [5]:

model.compile(loss=tf.keras.losses.MeanSquaredError(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005))

In [6]:
history = model.fit(
    x_train,y_train,
    validation_data=(x_test, y_test),
    batch_size=16,
    epochs=100
)

Epoch 1/100
74/74 [==============================] - 1s 5ms/step - loss: 38730801152.0000 - val_loss: 37947224064.0000
Epoch 2/100
74/74 [==============================] - 0s 2ms/step - loss: 37569511424.0000 - val_loss: 36347084800.0000
Epoch 3/100
74/74 [==============================] - 0s 2ms/step - loss: 34872135680.0000 - val_loss: 32214267904.0000
Epoch 4/100
74/74 [==============================] - 0s 2ms/step - loss: 28619929600.0000 - val_loss: 23953160192.0000
Epoch 5/100
74/74 [==============================] - 0s 2ms/step - loss: 19209449472.0000 - val_loss: 14043261952.0000
Epoch 6/100
74/74 [==============================] - 0s 3ms/step - loss: 11663775744.0000 - val_loss: 8258287616.0000
Epoch 7/100
74/74 [==============================] - 0s 2ms/step - loss: 8424787456.0000 - val_loss: 6514083840.0000
Epoch 8/100
74/74 [==============================] - 0s 2ms/step - loss: 7328260608.0000 - val_loss: 5792567808.0000
Epoch 9/100
74/74 [==============================] - 

In [7]:
test_loss = model.evaluate(x_test, y_test,)

9/9 [==============================] - 0s 2ms/step - loss: 3626268160.0000


In [8]:
p = model.predict(x_test)
from sklearn.metrics import mean_absolute_error

# assume y_true and y_pred are your ground truth and predicted values, respectively
mae = mean_absolute_error(y_test, p)

print(f"MAE: {mae}")
mse = tf.keras.losses.mean_squared_error(y_test, p)
print("MSE: ", mse)


9/9 [==============================] - 0s 1ms/step
MAE: 32991.733432583045
MSE:  tf.Tensor(
[9.3659402e+09 2.8355516e+10 1.1718428e+10 9.5933655e+09 9.0153687e+09
 2.2495461e+10 1.7807499e+10 7.4707302e+09 1.0770937e+11 6.2096660e+09
 1.3089889e+10 8.1558707e+09 6.1563756e+09 1.0079010e+10 6.1462764e+09
 6.4369295e+09 1.4161638e+10 6.7980733e+09 7.7002772e+09 6.7707453e+09
 8.3897595e+09 6.9650826e+09 7.3892946e+09 6.4396979e+09 6.2187776e+09
 6.2903665e+09 1.2164841e+10 6.4540841e+09 8.0487235e+09 7.6636237e+09
 7.2707676e+09 2.4246284e+10 8.0694799e+09 9.0761830e+09 6.7461033e+09
 8.3507840e+09 6.8702080e+09 6.1837000e+09 1.7608808e+10 1.2198842e+10
 1.0183550e+10 1.0563403e+10 6.8042061e+09 6.1514583e+09 2.7219397e+10
 1.3720499e+10 1.1183900e+10 7.9610665e+09 6.3716654e+09 1.9182998e+10
 6.5615729e+09 1.1660860e+10 7.9656479e+09 8.5612974e+09 1.4380971e+10
 6.2121677e+09 2.0531687e+10 6.5969270e+09 1.3670184e+10 6.3223076e+09
 6.1756979e+09 6.3764485e+09 6.3054556e+09 6.6656630e+09

In [9]:
test = pd.read_csv("https://raw.githubusercontent.com/Rahulsharma869/Housing-Price-Prediction/main/Datasets/test.csv")
test_data=test.loc[:,xtrain_col]
predicted_prices = model.predict(test_data)

# Create a dataframe with the predicted prices and the corresponding IDs
submission_df = pd.DataFrame({'Id': test.Id, 'SalePrice': predicted_prices.flatten()})

# Save the dataframe to a CSV file
submission_df.to_csv('submission.csv', index=False)
submission_df

46/46 [==============================] - 0s 1ms/step


,Id,SalePrice
0,1461,141695.984375
1,1462,170646.265625
2,1463,185429.359375
3,1464,180771.000000
4,1465,171490.953125
...,...,...
1454,2915,76439.812500
1455,2916,98173.000000
1456,2917,178402.359375
1457,2918,90945.125000


In [10]:
test = pd.read_csv("https://raw.githubusercontent.com/Rahulsharma869/Housing-Price-Prediction/main/Datasets/test.csv")
test_data=test.loc[:,xtrain_col]

In [13]:
predicted_prices = model.predict(test_data)

# Create a dataframe with the predicted prices and the corresponding IDs
submission_df = pd.DataFrame({'Id': test.Id, 'SalePrice': predicted_prices.flatten()})

# Save the dataframe to a CSV file
submission_df.to_csv('submission.csv', index=False)
submission_df

46/46 [==============================] - 0s 1ms/step


,Id,SalePrice
0,1461,141695.984375
1,1462,170646.265625
2,1463,185429.359375
3,1464,180771.000000
4,1465,171490.953125
...,...,...
1454,2915,76439.812500
1455,2916,98173.000000
1456,2917,178402.359375
1457,2918,90945.125000
